In [1]:
import pickle
import re
from tqdm import tqdm
import json

In [13]:
with open('C1_random_sentences.pickle', 'rb') as f:
    C1_random_sentences = pickle.load(f)

In [3]:
with open('C2_random_sentences.pickle', 'rb') as f:
    C2_random_sentences = pickle.load(f)

In [14]:
def create_jsonl_list(sentences):
    jsonl_list = []
    total_values = sum(len(lst) for lst in sentences.values())
    with tqdm(total=total_values) as pbar:
        for key, values in sentences.items():
            for value in values:
                diction = {}
                path = re.sub('ann_morphosyntax', 'text_structure', value[2])
                with open(path, 'r') as f:
                    unfinished = False
                    sentence = ''
                    for line in f:
                        sen = re.search(r'<u xml:id="(u-[0-9]*.[0-9]*)" who="#[A-Za-zżźćńółęąśŻŹĆĄŚĘŁÓŃ0-9,()?:!—%\"\'$#^&\*+=\[\]\{\}\/;. -]*">([A-Za-zżźćńółęąśŻŹĆĄŚĘŁÓŃ0-9,()?:!—%\"\'$#^&\*+=\[\]\{\}\/;. -„”]*)<\/u>', line)
                        if sen != None:
                            n = sen.group(1)
                            s = sen.group(2)
                            if unfinished == False:
                                if s.endswith('.') and value[1] in n:
                                    sentence += (' ' + s)
                                    separate = re.findall(r'[A-Za-zżźćńółęąśŻŹĆĄŚĘŁÓŃ0-9,()?:!—%\"\'$#^&\*+=\[\]\{\}\/;„” -]*\.*', sentence)
                                    for match in separate:
                                        if value[0] in match:
                                            m = re.sub(r'\s\([^)]*\)', '', match)
                                            diction['sentence'] = m.lstrip()
                                            i = 0
                                            w = 0
                                            for word in re.findall(r'\w+|[^\w\s]', m.lstrip()):
                                                if word != value[0] and word != ('„' or '”' or ':' or '.' or ',' or '%'):
                                                    w += 1
                                                    i += len(word)
                                                if word != value[0] and word == ('„' or '”' or ':' or '.' or ',' or '%'):
                                                    i += len(word)
                                                else:
                                                    w += 1
                                                    first = i + w - 1
                                                    break
                                            diction['first_index'] = first
                                            diction['last_index'] = first + len(value[0]) - 1
                                            diction['lemma'] = key
                                            jsonl_list.append(diction)
                                            pbar.update(1)
                                            break
                                    break
                                elif not s.endswith('.') and value[1] in n:
                                    sentence += s
                                    unfinished = True
                            else:
                                if s.endswith('.'):
                                    sentence += (' ' + s)
                                    separate = re.findall(r'[A-Za-zżźćńółęąśŻŹĆĄŚĘŁÓŃ0-9,()?:!—%\"\'$#^&\*+=\[\]\{\}\/;„” -]*\.*', sentence)
                                    for match in separate:
                                        if value[0] in match:
                                            m = re.sub(r'\s*\([^)]*\)', '', match)
                                            diction['sentence'] = m.lstrip()
                                            i = 0
                                            w = 0
                                            for word in re.findall(r'\w+|[^\w\s]', m.lstrip()):
                                                if word != value[0] and word != ('„' or '”' or ':' or '.' or ',' or '%'):
                                                    w += 1
                                                    i += len(word)
                                                if word != value[0] and word == ('„' or '”' or ':' or '.' or ',' or '%'):
                                                    i += len(word)
                                                else:
                                                    w += 1
                                                    first = i + w - 1
                                                    break
                                            diction['first_index'] = first
                                            diction['last_index'] = first + len(value[0]) - 1
                                            diction['lemma'] = key
                                            jsonl_list.append(diction)
                                            pbar.update(1)
                                            break
                                    break
                                else:
                                    sentence += (' ' + s)
                pbar.update(1)
    return jsonl_list

In [15]:
C1_jsonl_list = create_jsonl_list(C1_random_sentences)

642766it [06:39, 1608.10it/s]                            


In [16]:
sorted_list = sorted(C1_jsonl_list, key=lambda x: x["lemma"])
grouped_lists = {}
for d in sorted_list:
    lemma_value = d["lemma"]
    if lemma_value not in grouped_lists:
        grouped_lists[lemma_value] = [d]
    else:
        grouped_lists[lemma_value].append(d)

separate_lists = list(grouped_lists.values())

for group in separate_lists:
    json_object = json.dumps(group)
    filename = f"JSONL/{group[0]['lemma']}.jsonl"
    with open(filename, "w") as outfile:
        outfile.write(json_object)

In [21]:
C2_jsonl_list = create_jsonl_list(C2_random_sentences)

643183it [21:09, 506.68it/s]                            


In [22]:
sorted_list1 = sorted(C2_jsonl_list, key=lambda x: x["lemma"])
grouped_lists1 = {}
for d in sorted_list1:
    lemma_value = d["lemma"]
    if lemma_value not in grouped_lists1:
        grouped_lists1[lemma_value] = [d]
    else:
        grouped_lists1[lemma_value].append(d)

separate_lists1 = list(grouped_lists1.values())

for group in separate_lists1:
    json_object = json.dumps(group)
    filename = f"JSONL1/{group[0]['lemma']}.jsonl"
    with open(filename, "w") as outfile:
        outfile.write(json_object)